# Sysmon

Sysmon is a part of the SysInternals package, and is an expanded version of the windows event logs. It provides more details and a more granluar control system. The Events within Sysmon are still stored in the event logs here:

> Applications and Services Logs/Microsoft/WIndows/Sysmon/Operational

Once Sysmon is installed, it persists through restarts and its primary purpose is to integrate with a SIEM system. Thats not to say you cant monitor it internally, its just where it can really shine. Sysmon itself can record multiple event types (29ish at the time of writing) and you can write either include or exclude rules to filter which events are captured. These rules are stored in the Sysmon config file. Filtering at the monitor can reduce the amount of analysis needed at the SIEM end, however if the rules are not well created it can also hide malicious activity. If starting out, its great to start with a template and configure for your environment (once you know what youre doing). THM recommended [this one by SwiftOnSecurity](https://github.com/SwiftOnSecurity/sysmon-config/blob/master/sysmonconfig-export.xml).

<hr>

## Starting Sysmon

Once you have SysInternals installed, you will need to start sysmon (and initialise it). You can also initialise it with a config file. The below will launch SysMon with the SwiftOnSecurity config file. Launch this from the SysMon install location.
> Sysmon.exe -accepteula -i ..\Configuration\swift.xml

<hr>

## Config File

The config file is a .XML file. Within the XML is nested rules stored in groups, and each group is defined a matching type. The group then has its individual rules defined. The General structure is:

Sysmon Schemaversion
- Some global rules
  - Event Filtering
    - RuleGroup name = "" groupRelation="or"
      - ProcessCreate onmatch="exclude"
        - rule 1
        - rule 2
        - rule 3
      - /ProcessCreate
    - /RuleGroup


This is a very simple config file, which looks for the 'process created' event and excludes them from the logs. In this section we would have the 'trusted' applications, most likely the windows ones or BAU applications (such as chrome, office etc). Obviously some of these can still be malicious so should be carefully considered. In a 'full' config file, you would expect to have rules for all off the Event groups below.

### Best Practice for Rules

1. Exclude before Include
This isn't saying "write exclude rules before include rules", but rather that excluding cases (a whitelist) is better than including cases (a blacklist). In this way, any 'new' state is logged instead of being ignored. In other words, its a Fail-Safe state.

2. CLI Provides More Granular Control
It's harder (probably) to use CLI, but its also where the useful functions are, such as filtering. Use CLI where possible but dont let it get in the way of a complete ruleset. This brings us to....

3. Know Your Environment
Sounds obvious, but you cant filter bad data without knowing what good data is. If you're from the 'agile' mindset, then yes we can constantly improve but you should also ensure you have a good starting position. If not you are making things more difficult further down the track and things can be missed.
      
<hr>

## Event Groups

As above, the config file can filter multiple event groups, but which should we be looking for?

### 1: Process Creation
As above, this event can be used to filter out 'safe' processes or include known 'bad' processes. Likely we would want to exclude known, not include what may be unknown. Part of excluding 'safe' processes is that we can also identify if a fake process is run by being mispelt or programs being launched from a non-standard location. Below we are excluding chrome and windows media player.

### 3: Network Connection
This time we are looking for an application making a network connection, or known malicious ports. When we're red teaming, this is part of why we dont use standard ports even on the red side, it's too easy to pick up.

### 7: Image Loaded
It says image, but what it really means is a DLL being loaded by a process. As pretty much everything uses a .dll file. The below is an example of an "ImageLoaded" tag, but it also includes Image, SIgned and Signatures.

## 8: Remote Thread
This event occurs when a process tries to inject code into another process. Theres 2 ways to approach this event below. The first is looking for a specific memory address ending, and the second is looking for processes that doent also have a parent process when this is done. There are legitimate times this event can be triggered, but these two options are pretty red flags.

### 11: File Created

This one is a bit easier, if a file is created or overwritten/modified. You can add ransomware strings, known malicious file names (or just ones you dont want).

### 12,13,14: Registry Event
This is a change to the Windows registry. Theres many reasons malicious software would want to modify the registry, but its also a common event for 'safe' software. The below will looking for changes in a common malicious directory.

### 15: FileCreateStreamHash
Alternate data streams are a common way to hide files, it comes up in steganography quite often. Simply your 'file' can be 2 (or more) files, depending on how you call the file.

### 22: DNS Event
DNS queries are naturally part of a normal system running as expected, so its best to exclude trusted domains to keep things clean. Even so you will probably find this one can be quite noisy.

<hr>